In [1]:
1+8

9

In [18]:
print(df.columns.tolist())


['transaction_id', 'order_date', 'customer_id', 'product_id', 'product_name', 'category', 'subcategory', 'brand', 'original_price_inr', 'discount_percent', 'discounted_price_inr', 'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr', 'customer_city', 'customer_state', 'customer_tier', 'customer_spending_tier', 'customer_age_group', 'payment_method', 'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale', 'festival_name', 'customer_rating', 'return_status', 'order_month', 'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible', 'product_rating', 'revenue', 'year', 'month', 'month_year', 'qty']


In [5]:
import pandas as pd
import numpy as np

# =====================================================
# 1️⃣ LOAD DATA
# =====================================================
file_path = r'C:\project\amazon project\data\amazon_india_cleaned_final.csv'
df = pd.read_csv(file_path)

df.columns = df.columns.str.strip().str.lower()

# =====================================================
# 2️⃣ DATE & BASIC FEATURES
# =====================================================
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month
df['quarter'] = df['order_date'].dt.to_period('Q')
df['month_year'] = df['order_date'].dt.to_period('M')

# =====================================================
# 3️⃣ NUMERIC CLEANING
# =====================================================
num_cols = [
    'final_amount_inr','discount_percent','quantity',
    'delivery_charges','product_rating','discounted_price_inr',
    'delivery_days','customer_rating'
]

for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

df['revenue'] = df['final_amount_inr']
df['qty'] = df['quantity']

# =====================================================
# DASHBOARD OUTPUT
# =====================================================
print("="*70)
print("AMAZON INDIA – STRATEGIC  DASHBOARD (30 QUESTIONS)")
print("="*70)

# ================= EXECUTIVE (1–5) =================
print(f"\nQ1 Total Revenue: ₹{df['revenue'].sum():,.2f}")
print(f"Q2 Monthly Revenue Run Rate: ₹{df.groupby('month_year')['revenue'].sum().mean():,.2f}")
print(f"Q3 Geographic Coverage (States): {df['customer_state'].nunique()}")
print(f"Q4 Avg Discount (%): {df['discount_percent'].mean():.2f}")
print(f"Q5 Avg Order Value: ₹{df['revenue'].mean():,.2f}")

# ================= REVENUE (6–10) =================
print("\nQ6 Quarterly Revenue Trend:")
print(df.groupby('quarter')['revenue'].sum())

# -------------------------------
# Q7 FIX: Clean Category Names
# -------------------------------
df['category_clean'] = (
    df['category']
    .astype(str)
    .str.upper()
    .str.strip()
    .replace({
        'ELECTRONIC': 'ELECTRONICS',
        'ELECTRONICSS': 'ELECTRONICS'
    })
)

print("\nQ7 Top 3 Categories by Revenue (Cleaned):")
print(
    df.groupby('category_clean')['revenue']
      .sum()
      .nlargest(3)
)


print(f"\nQ8 Top Revenue City: {df.groupby('customer_city')['revenue'].sum().idxmax()}")

festival_rev = df[df['month'].isin([10,11])]['revenue'].sum()
print(f"Q9 Festival Revenue (Oct–Nov): ₹{festival_rev:,.2f}")

print(f"Q10 Avg Discounted Price: ₹{df['discounted_price_inr'].mean():,.2f}")

# ================= CUSTOMER (11–15) =================
print("\nQ11 Spending Tier Distribution:")
print(df['customer_spending_tier'].value_counts())

print(f"\nQ12 Top Acquisition City: {df['customer_city'].mode()[0]}")

# -------------------------------
# Q13 FIX: Normalize Prime Flag
# -------------------------------
df['prime_clean'] = (
    df['is_prime_member']
    .astype(str)
    .str.lower()
    .map({
        '1': 'Prime', 'true': 'Prime', 'yes': 'Prime',
        '0': 'Non-Prime', 'false': 'Non-Prime', 'no': 'Non-Prime'
    })
)

print("\nQ13 Prime vs Non-Prime Revenue (Cleaned):")
print(
    df.groupby('prime_clean')['revenue']
      .sum()
)


print(f"\nQ14 Avg Orders per Customer: {len(df)/df['customer_id'].nunique():.2f}")

print(f"Q15 Dominant Age Group: {df['customer_age_group'].mode()[0]}")

# ================= PRODUCT & INVENTORY (16–20) =================
print(f"\nQ16 Top Product by Units Sold: {df.groupby('product_name')['qty'].sum().idxmax()}")

print("\nQ17 Top 5 Brands:")
print(df['brand'].value_counts().head(5))

print(f"\nQ18 Total Units Sold: {df['qty'].sum()}")

print(f"Q19 Avg Product Rating: {df['product_rating'].mean():.2f}")

# -------------------------------
# Q20 FIX: Prime Eligible Revenue
# -------------------------------
df['prime_eligible_clean'] = (
    df['is_prime_eligible']
    .astype(str)
    .str.lower()
    .map({
        '1': 1, 'true': 1, 'yes': 1,
        '0': 0, 'false': 0, 'no': 0
    })
)

prime_eligible_revenue = df[df['prime_eligible_clean'] == 1]['revenue'].sum()

print(f"\nQ20 Prime Eligible Revenue (Corrected): ₹{prime_eligible_revenue:,.2f}")


# ================= OPERATIONS (21–25) =================
print(f"\nQ21 Avg Delivery Days: {df['delivery_days'].mean():.2f}")

print(f"Q22 Most Popular Payment Method: {df['payment_method'].mode()[0]}")

return_rate = df['return_status'].value_counts(normalize=True).get('Returned',0)*100
print(f"Q23 Return Rate: {return_rate:.2f}%")

print("\nQ24 Delivery Type Split:")
print(df['delivery_type'].value_counts())

print("\nQ25 Customer Rating Distribution:")
print(df['customer_rating'].value_counts().head(3))

# ================= ADVANCED (26–30) =================
print(f"\nQ26 Forecast Next Year (5% Growth): ₹{df['revenue'].sum()*1.05:,.2f}")

print(f"Q27 Top Subcategory: {df['subcategory'].mode()[0]}")

print("\nQ28 Revenue per Brand (Top 3):")
print(df.groupby('brand')['revenue'].mean().nlargest(3))

print(f"\nQ29 Peak Revenue Month: {df.groupby('month')['revenue'].sum().idxmax()}")

print("\nQ30  STATUS: ✅ ALL 30 QUESTIONS EXECUTED SUCCESSFULLY")

# =====================================================
# SAVE OUTPUT
# =====================================================
out_path = r'C:\project\amazon project\data\dashboard_master.csv'
df.to_csv(out_path, index=False)
print(f"\n✅ Master Dataset Saved → {out_path}")


AMAZON INDIA – STRATEGIC  DASHBOARD (30 QUESTIONS)

Q1 Total Revenue: ₹76,888,729,563.25
Q2 Monthly Revenue Run Rate: ₹526,127,289.51
Q3 Geographic Coverage (States): 15
Q4 Avg Discount (%): 17.42
Q5 Avg Order Value: ₹68,187.40

Q6 Quarterly Revenue Trend:
quarter
2015Q1    4.077824e+08
2015Q2    4.229215e+08
2015Q3    4.463431e+08
2015Q4    6.464558e+08
2016Q1    6.893188e+08
2016Q2    7.199785e+08
2016Q3    7.746665e+08
2016Q4    1.064507e+09
2017Q1    1.043816e+09
2017Q2    1.128685e+09
2017Q3    1.164253e+09
2017Q4    1.637609e+09
2018Q1    1.356784e+09
2018Q2    1.475876e+09
2018Q3    1.547387e+09
2018Q4    2.160327e+09
2019Q1    1.638057e+09
2019Q2    1.741300e+09
2019Q3    1.835097e+09
2019Q4    2.562242e+09
2020Q1    2.450260e+09
2020Q2    2.405044e+09
2020Q3    2.528359e+09
2020Q4    3.343450e+09
2021Q1    2.294747e+09
2021Q2    2.210407e+09
2021Q3    2.325881e+09
2021Q4    3.103942e+09
2022Q1    1.767105e+09
2022Q2    1.689428e+09
2022Q3    1.831248e+09
2022Q4    2.412532e+09